In [1]:
import datetime

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline, interp1d, BPoly, PPoly, CubicSpline

In [3]:
o_path = './data/2022_option_GSKGILD.csv'
option = pd.read_csv(o_path)

C:\Users\Sakura\AppData\Local\Temp\ipykernel_15420\2000232997.py:2: DtypeWarning: Columns (26,32) have mixed types. Specify dtype option on import or set low_memory=False.
  option = pd.read_csv(o_path)


In [4]:
option.shape

(360412, 38)

In [5]:
len(option)

360412

In [6]:
# AMSettlement
# 0 options on the security expire at the market close of the last trading day
# 1 – options on the security expire at the market open of the last trading day
# In other words, if an option is AM settled, as most cashsettled index option classes are, we use one less day than
# we use for PM-settled options to count days to expiration.
np.unique(option['am_settlement'], return_counts=True)

(array([0], dtype=int64), array([360412], dtype=int64))

In [7]:
call = option[option['cp_flag'] == 'C']
call = call[~call['impl_volatility'].isna()]
call = call[~call['last_date'].isna()]
call = call[call['am_settlement'] == 0]
call = call[['symbol', 'ticker', 'date', 'exdate', 'last_date', 'cp_flag', 'strike_price', 'best_bid', 
             'best_offer', 'volume', 'open_interest', 'impl_volatility', 'am_settlement', 
             'contract_size']]
# WRDS times stikes with 1000
call.loc[:, 'strike_price'] /= 1000
call.loc[:, 'mid_price'] = (call['best_bid'] + call['best_offer'])/2
call = call[call['open_interest'] > 0]
call = call.reset_index(drop=True)
df_len = call.shape[0]
for k in range(df_len):
    call.loc[k, 'maturity'] = (datetime.datetime.strptime(call.loc[k, 'exdate'], "%Y-%m-%d") -
                               datetime.datetime.strptime(call.loc[k, 'date'], "%Y-%m-%d")).days

In [8]:
call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93655 entries, 0 to 93654
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           93655 non-null  object 
 1   ticker           93655 non-null  object 
 2   date             93655 non-null  object 
 3   exdate           93655 non-null  object 
 4   last_date        93655 non-null  object 
 5   cp_flag          93655 non-null  object 
 6   strike_price     93655 non-null  float64
 7   best_bid         93655 non-null  float64
 8   best_offer       93655 non-null  float64
 9   volume           93655 non-null  int64  
 10  open_interest    93655 non-null  int64  
 11  impl_volatility  93655 non-null  float64
 12  am_settlement    93655 non-null  int64  
 13  contract_size    93655 non-null  int64  
 14  mid_price        93655 non-null  float64
 15  maturity         93655 non-null  float64
dtypes: float64(6), int64(4), object(6)
memory usage: 11.4+ MB


In [9]:
zero_cou = pd.read_csv('./data/2022_zeros.csv')
forward = pd.read_csv('./data/2022_forward_GSKGILD.csv')

In [10]:
max_days = call['maturity'].max() + 1.0
date = zero_cou['date'].unique()
date_len = date.shape[0]

In [11]:
max_days

500.0

In [12]:
interest_df = pd.DataFrame()
for i in range(date_len):
    x = zero_cou[zero_cou['date'] == date[i]]['days'].values
    y = zero_cou[zero_cou['date'] == date[i]]['rate'].values
    f = interp1d(x, y, fill_value=(y.min(), y.max()), bounds_error=False)
#     days_arr = np.arange(x.min(), max_days, 1.0)
    days_arr = np.arange(max_days)
    rate_arr = f(days_arr)
    df = pd.DataFrame({'days': days_arr, 'rate': rate_arr})
#     short_days = np.arange(0.0, x.min(), 1.0)
#     short_rate = zero_cou[(zero_cou['date'] == date[i]) & (zero_cou['days'] == x.min())]['rate'].values
#     short_df = pd.DataFrame({'days': short_days, 'rate': short_rate*np.ones_like(short_days)})
#     df = pd.concat([short_df, df], ignore_index=True)
    df.loc[:, 'date'] = date[i]
    interest_df = pd.concat([interest_df, df], ignore_index=True)

In [13]:
for i in range(call.shape[0]):
    cur_day = call.iloc[i]['date']
    maturity = call.iloc[i]['maturity']
    rate = interest_df[(interest_df['days']==maturity) & (interest_df['date']==cur_day)]['rate'].values
    call.loc[i, 'rf_rate'] = rate

In [14]:
forward['AMSettlement'].unique()

array([0], dtype=int64)

In [15]:
option['ticker'].unique(), forward['ticker'].unique()

(array(['GILD', 'GSK'], dtype=object), array(['GILD', 'GSK'], dtype=object))

In [16]:
for name in ['GILD', 'GSK']:
    for i in range(date.shape[0]):
        f_date = forward[(forward['date']==date[i]) & (forward['ticker'] == name)]['expiration'].unique()
        opt_date = call[(call['date']==date[i]) & (call['ticker'] == name)]['exdate'].unique()
        if not set(opt_date).issubset(opt_date):
            print('There are dates when option is expired but no forward price info.')

In [17]:
for i in range(call.shape[0]):
    name = call.iloc[i]['ticker']
    cur_day = call.iloc[i]['date']
    maturity = call.iloc[i]['exdate']
    am_set = call.iloc[i]['am_settlement']
    fwd_price = forward[(forward['date'] == cur_day) & (forward['expiration'] == maturity) & 
                        (forward['ticker'] == name)]['ForwardPrice'].values
    call.loc[i, 'forward'] = fwd_price

In [18]:
GILD = call[call['ticker'] == 'GILD']
GSK = call[call['ticker'] == 'GSK']

In [19]:
GILD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56206 entries, 0 to 87272
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           56206 non-null  object 
 1   ticker           56206 non-null  object 
 2   date             56206 non-null  object 
 3   exdate           56206 non-null  object 
 4   last_date        56206 non-null  object 
 5   cp_flag          56206 non-null  object 
 6   strike_price     56206 non-null  float64
 7   best_bid         56206 non-null  float64
 8   best_offer       56206 non-null  float64
 9   volume           56206 non-null  int64  
 10  open_interest    56206 non-null  int64  
 11  impl_volatility  56206 non-null  float64
 12  am_settlement    56206 non-null  int64  
 13  contract_size    56206 non-null  int64  
 14  mid_price        56206 non-null  float64
 15  maturity         56206 non-null  float64
 16  rf_rate          56206 non-null  float64
 17  forward     

In [20]:
GSK.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37449 entries, 46693 to 93654
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           37449 non-null  object 
 1   ticker           37449 non-null  object 
 2   date             37449 non-null  object 
 3   exdate           37449 non-null  object 
 4   last_date        37449 non-null  object 
 5   cp_flag          37449 non-null  object 
 6   strike_price     37449 non-null  float64
 7   best_bid         37449 non-null  float64
 8   best_offer       37449 non-null  float64
 9   volume           37449 non-null  int64  
 10  open_interest    37449 non-null  int64  
 11  impl_volatility  37449 non-null  float64
 12  am_settlement    37449 non-null  int64  
 13  contract_size    37449 non-null  int64  
 14  mid_price        37449 non-null  float64
 15  maturity         37449 non-null  float64
 16  rf_rate          37449 non-null  float64
 17  forward 

In [21]:
GILD.to_csv('./data/2022_GILDCall.csv', index=False)
GSK.to_csv('./data/2022_GSKCall.csv', index=False)

In [23]:
GSK[GSK['date']=='2023-01-31']['exdate'].unique()

array(['2023-02-03', '2023-02-10', '2023-02-17', '2023-02-24',
       '2023-03-03', '2023-03-10', '2023-03-17', '2023-05-19',
       '2023-06-16', '2023-08-18', '2024-01-19'], dtype=object)

In [24]:
GILD[GILD['date']=='2023-01-31']['exdate'].unique()

array(['2023-02-03', '2023-02-10', '2023-02-17', '2023-02-24',
       '2023-03-03', '2023-03-10', '2023-03-17', '2023-04-21',
       '2023-05-19', '2023-06-16', '2023-08-18', '2024-01-19'],
      dtype=object)